In [12]:
import warnings

import numpy as np
import pandas as pd
from google.cloud import storage

from scratch.compare_msi_output_utils import (
    get_terra_outputs,
    combine_dis_wavg,
    save_msi_outputs,
)

pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.max_info_columns", 30)
pd.set_option("display.max_info_rows", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_seq_items", None)
pd.set_option("display.width", 200)
pd.set_option("expand_frame_repr", True)
pd.set_option("mode.chained_assignment", "warn")

In [3]:
old_samples = get_terra_outputs(
    namespace="broad-firecloud-ccle", workspace="DepMap_WGS_CN"
)
new_samples = get_terra_outputs(
    namespace="broad-firecloud-ccle", workspace="omics_wgs_pipeline"
)

sample_ids = set(old_samples["sample_id"]).intersection(set(new_samples["sample_id"]))
old_samples = old_samples.loc[old_samples["sample_id"].isin(sample_ids)]
new_samples = new_samples.loc[new_samples["sample_id"].isin(sample_ids)]

In [4]:
storage_client = storage.Client(project="depmap-omics")
old_samples = save_msi_outputs(storage_client, old_samples, old_or_new="old")
new_samples = save_msi_outputs(storage_client, new_samples, old_or_new="new")

# Scores

Comparing old and new msisensor2 scores per sample

In [9]:
score_comp = old_samples[["sample_id", "msisensor2_score"]].merge(
    new_samples[["sample_id", "msisensor2_score"]],
    on="sample_id",
    how="inner",
    suffixes=("_old", "_new"),
)

score_comp

,sample_id,msisensor2_score_old,msisensor2_score_new
0,CDS-00Nrci,3.68,3.56
1,CDS-0bV15m,2.19,2.16
2,CDS-0e3PRe,2.39,2.41
3,CDS-0b4jFH,4.76,4.83
4,CDS-0JJKl3,3.19,3.19
5,CDS-0q5dcO,2.00,2.00
6,CDS-099jzP,2.87,2.82
7,CDS-0A4mDu,2.48,2.48
8,CDS-01m27W,2.41,2.35
9,CDS-051xn7,2.21,2.25


## Absolute deviation

In [10]:
score_med_abs_dev = np.abs(
    score_comp["msisensor2_score_new"] - score_comp["msisensor2_score_old"]
).median()
score_max_abs_dev = np.abs(
    score_comp["msisensor2_score_new"] - score_comp["msisensor2_score_old"]
).max()

print(f"Median absolute deviation: {round(score_med_abs_dev, 2)}")
print(f"Max absolute deviation: {round(score_max_abs_dev, 2)}")

Median absolute deviation: 0.03
Max absolute deviation: 0.12


# Repeats

Comparing old and new msisensor2 repeats per sample, per locus

In [13]:
with warnings.catch_warnings(action="ignore"):
    old_repeats = combine_dis_wavg(old_samples)
    new_repeats = combine_dis_wavg(new_samples)

repeats_comp = old_repeats.join(
    new_repeats, how="outer", lsuffix="_old", rsuffix="_new"
)

# make sure the outer join was effectively an inner one (i.e. indexes match
assert len(repeats_comp) == len(old_repeats) == len(new_repeats)

repeats_comp

CDS-00Nrci_old  CDS-0bV15m_old  CDS-0e3PRe_old  CDS-0b4jFH_old  CDS-0JJKl3_old  CDS-0q5dcO_old  CDS-099jzP_old  CDS-0A4mDu_old  CDS-01m27W_old  \
chr  loc       left_flank repeat right_flank                                                                                                                                                   
chr1 100077146 AAATC      15[T]  CAGAC             14.714286       15.000000       15.034483            15.0       15.030303       15.000000       15.000000       15.000000       15.000000   
     100218786 AAAAA      12[T]  ACTAA             12.000000       12.000000       11.956522            12.0       12.000000       12.032258       11.894737       11.970588       12.000000   
     10072379  ATCGC      10[T]  ATTGC              9.892857       10.000000       10.000000            10.0       10.000000       10.000000       10.045455        9.976744        9.961538   
     100990636 CTATT      10[A]  GATAC             10.000000       10.000000       10.000000            10.0       10.000000       10.030303       10.000000       10.000000       10.000000   
     103107961 ATTGC      14[T]  CTTGA             13.813953       14.000000       13.916667            14.0       14.000000       14.000000       14.000000       13.916667       14.000000   
...                                                      ...             ...             ...             ...             ...             ...             ...             ...             ...   
chr9 94301443  AATGG      11[A]  TGTTG             10.902439       10.972973       11.000000            11.0       11.000000       11.000000       11.000000       11.030303       11.000000   
     94664048  GGAAT      13[A]  GCCAG             12.857143       13.000000       13.000000            13.0       13.000000       13.000000       13.361702       13.274510       13.000000   
     95012920  TTTTG      12[T]  AACGT             12.100000       12.000000       12.000000            12.0       12.000000       12.000000       11.960000       12.000000       12.000000   
     96645797  TCTCG      14[A]  GGAAA             13.952381       14.000000       14.000000            14.0       14.000000       14.000000       14.000000       14.000000       14.000000   
     97439221  TTTAC      13[T]  AATAT             12.928571       13.064516       13.000000            13.0       12.948718       13.000000       13.000000       13.000000       13.000000   

                                              CDS-051xn7_old  CDS-0Eax8o_old  CDS-00Nrci_new  CDS-0q5dcO_new  CDS-0e3PRe_new  CDS-0b4jFH_new  CDS-0JJKl3_new  CDS-0bV15m_new  CDS-051xn7_new  \
chr  loc       left_flank repeat right_flank                                                                                                                                                   
chr1 100077146 AAATC      15[T]  CAGAC             15.000000       14.962963       14.714286       15.000000       15.034483            15.0       15.030303       15.000000       15.000000   
     100218786 AAAAA      12[T]  ACTAA             12.000000       11.958333       12.000000       12.032258       11.956522            12.0       12.000000       12.000000       12.000000   
     10072379  ATCGC      10[T]  ATTGC              9.972222       10.000000        9.888889       10.000000       10.000000            10.0       10.000000       10.000000        9.972222   
     100990636 CTATT      10[A]  GATAC             10.000000       10.000000       10.000000       10.030303       10.000000            10.0       10.000000       10.000000       10.000000   
     103107961 ATTGC      14[T]  CTTGA             14.000000       14.000000       13.809524       14.000000       13.916667            14.0       14.000000       14.000000       14.000000   
...                                                      ...             ...             ...             ...             ...             ...             ...             ...             ...   
chr9 94301443

In [14]:
id_vars = ["chr", "loc", "left_flank", "repeat", "right_flank"]

old_repeats_long = old_repeats.reset_index().melt(
    id_vars=id_vars,
    var_name="sample_id",
    value_name="wavg",
)

new_repeats_long = new_repeats.reset_index().melt(
    id_vars=id_vars,
    var_name="sample_id",
    value_name="wavg",
)

long_comp = old_repeats_long.merge(
    new_repeats_long,
    on=[*id_vars, "sample_id"],
    how="inner",
    suffixes=("_old", "_new"),
)

long_comp["abs_dev"] = np.abs(long_comp["wavg_old"] - long_comp["wavg_new"])
long_comp["abs_dev_rank"] = long_comp["abs_dev"].rank(ascending=False, method="min")

long_comp

,chr,loc,left_flank,repeat,right_flank,sample_id,wavg_old,wavg_new,abs_dev,abs_dev_rank
0,chr1,100077146,AAATC,15[T],CAGAC,CDS-00Nrci,14.714286,14.714286,0.000000,2457.0
1,chr1,100218786,AAAAA,12[T],ACTAA,CDS-00Nrci,12.000000,12.000000,0.000000,2457.0
2,chr1,10072379,ATCGC,10[T],ATTGC,CDS-00Nrci,9.892857,9.888889,0.003968,1608.0
3,chr1,100990636,CTATT,10[A],GATAC,CDS-00Nrci,10.000000,10.000000,0.000000,2457.0
4,chr1,103107961,ATTGC,14[T],CTTGA,CDS-00Nrci,13.813953,13.809524,0.004430,1562.0
...,...,...,...,...,...,...,...,...,...,...
31103,chr9,94301443,AATGG,11[A],TGTTG,CDS-0Eax8o,11.019231,11.019231,0.000000,2457.0
31104,chr9,94664048,GGAAT,13[A],GCCAG,CDS-0Eax8o,13.000000,13.000000,0.000000,2457.0
31105,chr9,95012920,TTTTG,12[T],AACGT,CDS-0Eax8o,12.000000,12.000000,0.000000,2457.0
31106,chr9,96645797,TCTCG,14[A],GGAAA,CDS-0Eax8o,14.000000,14.000000,0.000000,2457.0


## Absolute deviation

In [16]:
overall_med_abs_dev = long_comp["abs_dev"].median()
print(f"Overall median absolute deviation: {round(overall_med_abs_dev, 2)}")

Overall median absolute deviation: 0.0
